## Multi Blob Data Set 

Kmeans

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import sklearn.preprocessing as prep
from sklearn.datasets import make_blobs
from plotnine import *
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline

def display_cluster(axes,k,z,X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[km.labels_==i,0],X[km.labels_==i,1],c = color[i],alpha = alpha,s=s)
            axes[k][z].scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)
            axes[k][z].title.set_text('K = '+str(i+1))
            plt.rcParams['figure.figsize'] = [12, 8]
            
centers = [(-3, -3), (0, 0), (5,2.5),(-1, 4), (4, 6), (9,7)]
Multi_blob_Data, y = make_blobs(n_samples=[100,150, 300, 400,300, 200], n_features=2, cluster_std=[1.3,0.6, 1.2, 1.7,0.9,1.7],
                  centers=centers, shuffle=False, random_state=42)

kmeans=[]
inertias=[]
score=[]

k=0
z=0

for j in range(1,18):
    
    kmeans.append(KMeans(n_clusters=j))
    kmeans[j-1]=kmeans[j-1].fit(Multi_blob_Data)
    inertias.append(kmeans[j-1].inertia_)
    
    if(j>1 and j<18):
        score.append(silhouette_score(Multi_blob_Data, kmeans[j-1].labels_, metric='euclidean'))
        
Best_score=max(score)
K_of_Best_score=2+score.index(Best_score)

plt.figure(1)       
plt.plot(range(1,18), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('inertias')

plt.figure(2)       
plt.plot(range(2,18), score, marker='o')
plt.title('Silhouette score versus K')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')

f = plt.figure(3)    
f, axes = plt.subplots(nrows = 2, ncols = 3, sharex=True, sharey = True)
plt.rcParams['figure.figsize'] = [12, 8]

for j in range(1,7):
    
    display_cluster(axes,k,z,Multi_blob_Data,kmeans[j-1],num_clusters=j)
    
    if j == 3:
        k=k+1
        z=0
    else:
        z=z+1
        

print('Best K = '+str(K_of_Best_score)+' silhouette score= '+str(Best_score))

 Hierarchal Clustering

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc

from sklearn.datasets import make_blobs

# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(axes,k,z,parameter,X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 1
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[km==i,0],X[km==i,1],cmap = 'plasma',alpha = alpha,s=s)
            axes[k][z].title.set_text(parameter)
            plt.rcParams['figure.figsize'] = [12, 8]
            
centers = [(-3, -3), (0, 0), (5,2.5),(-1, 4), (4, 6), (9,7)]
Multi_blob_Data, y = make_blobs(n_samples=[100,150, 300, 400,300, 200], n_features=2, cluster_std=[1.3,0.6, 1.2, 1.7,0.9,1.7],
                  centers=centers, shuffle=False, random_state=42)

hc=[]
score=[]
parameters=[]
score_parameters=[]
clusters=[]
dist=[]
j=0
t=0
for aff in ['euclidean','manhattan','cosine']:
    for link in ['average','single']:
        z=0
        k=0
        j=j+1
        f = plt.figure(j)    
        f, axes = plt.subplots(nrows = 6, ncols = 3, sharex=True, sharey = True)
        f.delaxes(axes[0][2])
        f.delaxes(axes[5][2])
        for i in range(1,18):
            hc.append(AgglomerativeClustering(distance_threshold=float(i),affinity=aff,linkage=link,n_clusters=None))
            hc[t]=hc[t].fit_predict(Multi_blob_Data)
            clusters.append(len(np.unique(hc[t])))
            parameters.append(aff+' '+link+' clusters='+str(clusters[t]))#+' '+'clusters='+str(clusters[t])+' Distance threshold = '+str(float(i))
            display_cluster(axes,k,z,parameters[len(parameters)-1],Multi_blob_Data,hc[t],num_clusters=clusters[t])
            if(len(np.unique(hc[t]))>1):
                dist.append(float(i))
                score.append(silhouette_score(Multi_blob_Data, hc[t], metric=aff))
                score_parameters.append(parameters[len(parameters)-1])
            
            t=t+1
            
            if i % 3==0:
                k=k+1
                z=0
            else:
                z=z+1
Best_score=max(score)
Best_dist=dist[score.index(Best_score)]
Best_score_parameter=score_parameters[score.index(Best_score)]
print(Best_score_parameter+' score= '+str(Best_score)+' With distance thresholed = '+str(Best_dist))

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_av = shc.dendrogram(shc.linkage( Multi_blob_Data, method='average'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_si = shc.dendrogram(shc.linkage( Multi_blob_Data, method='single'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

DBSCAN

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import make_blobs
from plotnine import *

from sklearn.cluster import DBSCAN

from sklearn import metrics

from sklearn.metrics import silhouette_score

%matplotlib inline

def display_cluster(X,km=[]):
    plt.scatter(Multi_blob_Data[:, 0], Multi_blob_Data[:,1], c = km.labels_, cmap= "plasma",alpha = 0.5,s = 20)
            
centers = [(-3, -3), (0, 0), (5,2.5),(-1, 4), (4, 6), (9,7)]
Multi_blob_Data, y = make_blobs(n_samples=[100,150, 300, 400,300, 200], n_features=2, cluster_std=[1.3,0.6, 1.2, 1.7,0.9,1.7],
                  centers=centers, shuffle=False, random_state=42)
score=[]
epsilon=[]
clusters=[]
min_samp=[]

for e in np.arange(0.1, 3.1, 0.1):
    for mini in range(5,26):
        dbscan = DBSCAN(eps =e, min_samples = mini)
        dbscan= dbscan.fit(Multi_blob_Data)
        Number_of_clusters=len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
        if Number_of_clusters > 1:
            epsilon.append(e)
            clusters.append(Number_of_clusters)
            min_samp.append(mini)
            score.append(metrics.silhouette_score(Multi_blob_Data, dbscan.labels_))
            
plt.rcParams['figure.figsize'] = [12, 8] 
plt.figure(1)      
plt.plot(epsilon, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('epsilon')
plt.ylabel('Silhouette score')

plt.figure(2)       
plt.plot(min_samp, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('min_samp')
plt.ylabel('Silhouette score')


plt.figure(3)  
best_score=max(score)
g=score.index(best_score)
display_cluster(Multi_blob_Data,DBSCAN(eps =epsilon[g], min_samples = min_samp[g]).fit(Multi_blob_Data))
Best_score_parameters='Epsilon = '+str(epsilon[g])+' Min_samples = '+str(min_samp[g])
print(Best_score_parameters+' with score = '+str(best_score))

## iris data set 

Kmeans

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import load_iris
from plotnine import *  
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
%matplotlib inline


iris_data=load_iris()   #loading iris dataset from sklearn.datasets\
iris_df = pd.DataFrame(iris_data.data) #creating dataframe


def display_cluster(axes,k,z,labels,X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[labels==i][0],X[labels==i][1],c = color[i],alpha = alpha,s=s)
            axes[k][z].scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)
            axes[k][z].title.set_text('K = '+str(i+1))

kmeans=[]
inertias=[]
score=[]
labels = []

k=0
z=0

for j in range(1,18):
    
    kmeans.append(KMeans(n_clusters=j,init = 'k-means++',   max_iter = 100, n_init = 10, random_state = 0))
    kmeans[j-1]=kmeans[j-1].fit(iris_df)
    inertias.append(kmeans[j-1].inertia_)
    labels.append(kmeans[j-1].predict(iris_df))
    if(j>1 and j<18):
        score.append(silhouette_score(iris_df, kmeans[j-1].labels_, metric='euclidean'))
        
Best_score=max(score)
K_of_Best_score=2+score.index(Best_score)

plt.figure(1)       
plt.plot(range(1,18), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('inertias')

plt.figure(2)       
plt.plot(range(2,18), score, marker='o')
plt.title('Silhouette score versus K')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')

f = plt.figure(3)    
f, axes = plt.subplots(nrows = 2, ncols = 3, sharex=True, sharey = True)
plt.rcParams['figure.figsize'] = [12, 8]

for j in range(1,7):
    
    display_cluster(axes,k,z,labels[j-1],iris_df,kmeans[j-1],num_clusters=j)
    
    if j == 3:
        k=k+1
        z=0
    else:
        z=z+1
        

print('Best K = '+str(K_of_Best_score)+' score= '+str(Best_score))

After Normalization

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris
from plotnine import *  
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import silhouette_score
%matplotlib inline


iris_data=load_iris()   #loading iris dataset from sklearn.datasets\
iris_df = pd.DataFrame(iris_data.data) #creating dataframe
iris_df=preprocessing.normalize(iris_df)
iris_df = pd.DataFrame(iris_df) #creating dataframe

def display_cluster(axes,k,z,labels,X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[labels==i][0],X[labels==i][1],c = color[i],alpha = alpha,s=s)
#             axes[k][z].scatter(km.cluster_centers_[i][0],km.cluster_centers_[i][1],c = color[i], marker = 'x', s = 100)
            axes[k][z].title.set_text('K = '+str(i+1))

kmeans=[]
inertias=[]
score=[]
labels = []

k=0
z=0

for j in range(1,18):
    
    kmeans.append(KMeans(n_clusters=j,init = 'k-means++',   max_iter = 100, n_init = 10, random_state = 0))
    kmeans[j-1]=kmeans[j-1].fit(iris_df)
    inertias.append(kmeans[j-1].inertia_)
    labels.append(kmeans[j-1].predict(iris_df))
    if(j>1 and j<18):
        score.append(silhouette_score(iris_df, kmeans[j-1].labels_, metric='euclidean'))
        
Best_score=max(score)
K_of_Best_score=2+score.index(Best_score)

plt.figure(1)       
plt.plot(range(1,18), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('inertias')

plt.figure(2)       
plt.plot(range(2,18), score, marker='o')
plt.title('Silhouette score versus K')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')

f = plt.figure(3)    
f, axes = plt.subplots(nrows = 2, ncols = 3, sharex=True, sharey = True)
plt.rcParams['figure.figsize'] = [12, 8]

for j in range(1,7):
    
    display_cluster(axes,k,z,labels[j-1],iris_df,kmeans[j-1],num_clusters=j)
    
    if j == 3:
        k=k+1
        z=0
    else:
        z=z+1
        

print('Best K = '+str(K_of_Best_score)+' score= '+str(Best_score))

Hierarchal Clustering

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
from sklearn.datasets import load_iris


# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(axes,k,z,parameter,X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[km.labels_==i][0],X[km.labels_==i][1],cmap = 'plasma',alpha = alpha,s=s)
            axes[k][z].title.set_text(parameter)
            plt.rcParams['figure.figsize'] = [12, 8]
            
iris_data=load_iris()
iris_df = pd.DataFrame(iris_data.data)

hc=[]
score=[]
parameters=[]
score_parameters=[]
clusters=[]
dist=[]
j=0
t=0
n=0
for aff in ['euclidean','manhattan','cosine']:
    for link in ['average','single']:
        z=0
        k=0
        j=j+1
        f = plt.figure(j)    
        f, axes = plt.subplots(nrows = 6, ncols = 3, sharex=True, sharey = True)
#         f.delaxes(axes[0][2])
#         f.delaxes(axes[5][2])
        for i in np.arange(0.5, 8.1,0.5):
            hc.append(AgglomerativeClustering(distance_threshold=i,affinity=aff,linkage=link,n_clusters=None))
            hc[t]=hc[t].fit(iris_df)
            clusters.append(hc[t].n_clusters_)
            parameters.append(aff+' '+link+' clusters='+str(clusters[t]))
            display_cluster(axes,k,z,parameters[len(parameters)-1],iris_df,hc[t],num_clusters=clusters[t])
            if(hc[t].n_clusters_>1):
                dist.append(i)
                score.append(silhouette_score(iris_df, hc[t].labels_, metric=aff))
                score_parameters.append(parameters[len(parameters)-1])
            
            t=t+1
            
            if n % 3==0:
                k=k+1
                z=0
            else:
                z=z+1
            n=n+1
Best_score=max(score)
Best_dist=dist[score.index(Best_score)]
Best_score_parameter=score_parameters[score.index(Best_score)]
print(Best_score_parameter+' score= '+str(Best_score)+' With distance thresholed = '+str(Best_dist))

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_av = shc.dendrogram(shc.linkage( iris_df, method='average'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_si = shc.dendrogram(shc.linkage( iris_df, method='single'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

After Normalization

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
from sklearn.datasets import load_iris


# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(axes,k,z,parameter,X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 20
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[km.labels_==i][0],X[km.labels_==i][1],cmap = 'plasma',alpha = alpha,s=s)
            axes[k][z].title.set_text(parameter)
            plt.rcParams['figure.figsize'] = [12, 8]
            
iris_data=load_iris()
iris_df = pd.DataFrame(iris_data.data)
iris_df = preprocessing.normalize(iris_df)
iris_df = pd.DataFrame(iris_df)

hc=[]
score=[]
parameters=[]
score_parameters=[]
clusters=[]
dist=[]
j=0
t=0
n=0
for aff in ['euclidean','manhattan','cosine']:
    for link in ['average','single']:
        z=0
        k=0
        j=j+1
        f = plt.figure(j)    
        f, axes = plt.subplots(nrows = 6, ncols = 3, sharex=True, sharey = True)
#         f.delaxes(axes[0][2])
#         f.delaxes(axes[5][2])
        for i in np.arange(0.5, 8.1,0.5):
            hc.append(AgglomerativeClustering(distance_threshold=i,affinity=aff,linkage=link,n_clusters=None))
            hc[t]=hc[t].fit(iris_df)
            clusters.append(hc[t].n_clusters_)
            parameters.append(aff+' '+link+' clusters='+str(clusters[t]))
            display_cluster(axes,k,z,parameters[len(parameters)-1],iris_df,hc[t],num_clusters=clusters[t])
            if(hc[t].n_clusters_>1):
                dist.append(i)
                score.append(silhouette_score(iris_df, hc[t].labels_, metric=aff))
                score_parameters.append(parameters[len(parameters)-1])
            
            t=t+1
            
            if n % 3==0:
                k=k+1
                z=0
            else:
                z=z+1
            n=n+1
Best_score=max(score)
Best_dist=dist[score.index(Best_score)]
Best_score_parameter=score_parameters[score.index(Best_score)]
print(Best_score_parameter+' score= '+str(Best_score)+' With distance thresholed = '+str(Best_dist))

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_av = shc.dendrogram(shc.linkage( iris_df, method='average'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_si = shc.dendrogram(shc.linkage( iris_df, method='single'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

DBSCAN

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from plotnine import *   
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.metrics import silhouette_score
%matplotlib inline

def display_cluster(X,km=[]):
    plt.scatter(x[:, 0], x[:,1], c = km.labels_, cmap= "plasma",alpha = 0.5,s = 20)
            
iris_data=load_iris()
iris_df = pd.DataFrame(iris_data.data)

score=[]
epsilon=[]
clusters=[]
min_samp=[]

for e in np.arange(0.1, 3.1, 0.1):
    for mini in range(5,26):
        dbscan = DBSCAN(eps =e, min_samples = mini)
        dbscan=dbscan.fit(iris_df)
        Number_of_clusters=len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
        if Number_of_clusters > 1:
            epsilon.append(e)
            clusters.append(Number_of_clusters)
            min_samp.append(mini)
            score.append(metrics.silhouette_score(iris_df, dbscan.labels_))
            
plt.figure(1)  
plt.rcParams['figure.figsize'] = [12, 8]     
plt.plot(epsilon, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('epsilon')
plt.ylabel('Silhouette score')

plt.figure(2)       
plt.plot(min_samp, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('min_samp')
plt.ylabel('Silhouette score')

plt.figure(3) 

best_score=max(score)
g=score.index(best_score)
display_cluster(iris_df,DBSCAN(eps =epsilon[g], min_samples = min_samp[g]).fit(iris_df))
Best_score_parameters='Epsilon = '+str(epsilon[g])+' Min_samples = '+str(min_samp[g])
print(Best_score_parameters+' Score= '+str(best_score))

After Normalization

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from plotnine import *   
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import silhouette_score
%matplotlib inline

def display_cluster(X,km=[]):
    plt.scatter(X[:, 0], X[:,1], c = km.labels_, cmap= "plasma",alpha = 0.5,s = 20)
            
iris_data=load_iris()
iris_df = pd.DataFrame(iris_data.data)
iris_df = preprocessing.normalize(iris_df)


score=[]
epsilon=[]
clusters=[]
min_samp=[]

for e in np.arange(0.1, 3.1, 0.1):
    for mini in range(5,26):
        dbscan = DBSCAN(eps =e, min_samples = mini)
        dbscan=dbscan.fit(iris_df)
        Number_of_clusters=len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
        if Number_of_clusters > 1:
            epsilon.append(e)
            clusters.append(Number_of_clusters)
            min_samp.append(mini)
            score.append(metrics.silhouette_score(iris_df, dbscan.labels_))
            
plt.figure(1)  
plt.rcParams['figure.figsize'] = [12, 8]     
plt.plot(epsilon, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('epsilon')
plt.ylabel('Silhouette score')

plt.figure(2)       
plt.plot(min_samp, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('min_samp')
plt.ylabel('Silhouette score')

plt.figure(3) 

best_score=max(score)
g=score.index(best_score)
display_cluster(iris_df,DBSCAN(eps =epsilon[g], min_samples = min_samp[g]).fit(iris_df))
Best_score_parameters='Epsilon = '+str(epsilon[g])+' Min_samples = '+str(min_samp[g])
print(Best_score_parameters+' Score= '+str(best_score))

## Customer dataset

Kmeans

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import pandas as pd
from plotnine import *  
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics import silhouette_score
%matplotlib inline


df= pd.read_csv('Customer data.csv', index_col = 0)
# df_data = pd.DataFrame(df) #creating dataframe
df_data=preprocessing.normalize(df)
df_data = pd.DataFrame(df_data) #creating dataframe

def display_cluster(p,o,axes,k,z,labels,X,km=[],num_clusters=0):
    color = 'brgcmyk'
    alpha = 0.5
    s = 20
    y=[0,1,2,3,4,5,6]
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[labels==i][y[p]],X[labels==i][y[o]],c = color[i],alpha = alpha,s=s)
            axes[k][z].scatter(km.cluster_centers_[i][y[p]],km.cluster_centers_[i][y[o]],cmap = 'plasma', marker = 'x', s = 100)
            axes[k][z].title.set_text('K = '+str(i+1))

kmeans=[]
inertias=[]
score=[]
labels = []


for j in range(1,18):
    
    kmeans.append(KMeans(n_clusters=j,init = 'k-means++',   max_iter = 100, n_init = 10, random_state = 0))
    kmeans[j-1]=kmeans[j-1].fit(df_data)
    inertias.append(kmeans[j-1].inertia_)
    labels.append(kmeans[j-1].predict(df_data))
    if(j>1 and j<18):
        score.append(silhouette_score(df_data, kmeans[j-1].labels_, metric='euclidean'))
        
Best_score=max(score)
K_of_Best_score=2+score.index(Best_score)

plt.figure(1)       
plt.plot(range(1,18), inertias, marker='o')
plt.title('Elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('inertias')

plt.figure(2)       
plt.plot(range(2,18), score, marker='o')
plt.title('Silhouette score versus K')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')



for p in range (0,7):
    for o in range (0,7):
        if(p!=o):
            f = plt.figure(t+3)    
            f, axes = plt.subplots(nrows = 2, ncols = 3, sharex=True, sharey = True)
            plt.rcParams['figure.figsize'] = [12, 8]
            k=0
            z=0
            for j in range(1,7):

                display_cluster(p,o,axes,k,z,labels[j-1],df_data,kmeans[j-1],num_clusters=j)

                if j == 3:
                    k=k+1
                    z=0
                else:
                    z=z+1
            t=t+1

        

print('Best K = '+str(K_of_Best_score)+' score= '+str(Best_score))

Hierarchal Clustering

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score
import scipy.cluster.hierarchy as shc
from sklearn.datasets import load_iris


# helper function that allows us to display data in 2 dimensions an highlights the clusters
def display_cluster(p,o,axes,k,z,parameter,X,km=[],num_clusters=0):
    color = 'brgcmyk'  #List colors
    alpha = 0.5  #color obaque
    s = 20
    y=[0,1,2,3,4,5,6]
    if num_clusters == 0:
        plt.scatter(X[:,0],X[:,1],c = color[0],alpha = alpha,s = s)
    else:
        for i in range(num_clusters):
            axes[k][z].scatter(X[km.labels_==i][y[p]],X[km.labels_==i][y[o]],cmap = 'plasma',alpha = alpha,s=s)
            axes[k][z].title.set_text(parameter)
            plt.rcParams['figure.figsize'] = [12, 8]
            
df= pd.read_csv('Customer data.csv', index_col = 0)
df_data = pd.DataFrame(df) #creating dataframe
df_data=preprocessing.normalize(df_data)
df_data = pd.DataFrame(df_data) #creating dataframe

hc=[]
score=[]
parameters=[]
score_parameters=[]
clusters=[]
dist=[]
j=0
t=0
n=0
tt=0
for aff in ['euclidean','manhattan','cosine']:
    for link in ['average','single']:
        z=0
        k=0
        j=j+1
        f = plt.figure(j)    
        f, axes = plt.subplots(nrows = 4, ncols = 3, sharex=True, sharey = True)
        f.delaxes(axes[0][1])
        f.delaxes(axes[0][2])
        f.delaxes(axes[3][2])
        for i in np.arange(0.00001,0.0001,0.00001):
            hc.append(AgglomerativeClustering(distance_threshold=i,affinity=aff,linkage=link,n_clusters=None))
            hc[t]=hc[t].fit(df_data)
            clusters.append(hc[t].n_clusters_)
            parameters.append(aff+' '+link+' clusters='+str(clusters[t]))
            for p in range (0,7):
                for o in range (0,7):
                    if(p!=o):
                        f = plt.figure(tt+3)    
                        f, axes = plt.subplots(nrows = 2, ncols = 3, sharex=True, sharey = True)
                        plt.rcParams['figure.figsize'] = [12, 8]
                        k=0
                        z=0
                        for j in range(1,7):

                            display_cluster(p,o,axes,k,z,parameters[len(parameters)-1],df_data,hc[t],num_clusters=clusters[t])

                            if j == 3:
                                k=k+1
                                z=0
                            else:
                                z=z+1
                        tt=tt+1
            
            if(hc[t].n_clusters_>1):
                dist.append(i)
                score.append(silhouette_score(df_data, hc[t].labels_, metric=aff))
                score_parameters.append(parameters[len(parameters)-1])
            
            t=t+1
            
            if n % 3==0:
                k=k+1
                z=0
            else:
                z=z+1
            n=n+1
Best_score=max(score)
Best_dist=dist[score.index(Best_score)]
Best_score_parameter=score_parameters[score.index(Best_score)]
print(Best_score_parameter+' score= '+str(Best_score)+' With distance thresholed = '+str(Best_dist))

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_av = shc.dendrogram(shc.linkage( df_data, method='average'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]  
plt.title("Dendrograms")
dendrogram_av = shc.dendrogram(shc.linkage( df_data, method='single'))
plt.axhline(y=Best_dist, color='r', linestyle='--')

DBSCAN

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from plotnine import *   
from sklearn.cluster import DBSCAN
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import silhouette_score
%matplotlib inline

def display_cluster(X,km=[]):
    z=[0,1,2,3,4,5,6]
    for y in range(0,7):
        for y2 in range(0,7):
            if(y!=y2):
                plt.scatter(X[:, z[y]], X[:,z[y2]], c = km.labels_, cmap= "plasma",alpha = 0.5,s = 20)
            
df= pd.read_csv('Customer data.csv', index_col = 0)
df_data = pd.DataFrame(df) #creating dataframe
df_data=preprocessing.normalize(df_data)


score=[]
epsilon=[]
clusters=[]
min_samp=[]

for e in np.arange(0.00001,0.0001, 0.00001):
    for mini in range(5,26):
        dbscan = DBSCAN(eps =e, min_samples = mini)
        dbscan=dbscan.fit(df_data)
        Number_of_clusters=len(set(dbscan.labels_)) - (1 if -1 in dbscan.labels_ else 0)
        if Number_of_clusters > 1:
            epsilon.append(e)
            clusters.append(Number_of_clusters)
            min_samp.append(mini)
            score.append(metrics.silhouette_score(df_data, dbscan.labels_))
            
plt.figure(1)  
plt.rcParams['figure.figsize'] = [12, 8]     
plt.plot(epsilon, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('epsilon')
plt.ylabel('Silhouette score')

plt.figure(2)       
plt.plot(min_samp, score, marker='o')
plt.title('Silhouette score versus epsilon')
plt.xlabel('min_samp')
plt.ylabel('Silhouette score')

plt.figure(3) 

best_score=max(score)
g=score.index(best_score)
display_cluster(df_data,DBSCAN(eps =epsilon[g], min_samples = min_samp[g]).fit(df_data))
Best_score_parameters='Epsilon = '+str(epsilon[g])+' Min_samples = '+str(min_samp[g])
print(Best_score_parameters+' Score= '+str(best_score))